In [1]:
import requests
import lxml.html as lh
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
data = []
count = 1
for year in range(1980,2020):
    website_url = requests.get('https://www.basketball-reference.com/leagues/NBA_'+str(year)+'_per_game.html')
    soup = BeautifulSoup(website_url.content,'lxml')
    soup.prettify()
    My_table = soup.find('table',{'id':"per_game_stats"})
    tabledata = My_table.findAll('td')
    for cell in tabledata:
        count = count+1
        if (count%29 == 2):
            data.append(str(year))
        data.append(cell.get_text())

In [3]:
import numpy as np
data2 = np.array(data)
refined = np.reshape(data2, (-1, 30))

In [4]:
tableheader = My_table.findAll('th')
headers = []
count = 0
for item in tableheader:
    if (count < 30):
        headers.append(item.get_text())
        count=count+1
headers.remove('Rk')
headers = ['Year'] + headers
print(headers)

['Year', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']


In [13]:
import pandas as pd
df1 = pd.DataFrame(refined, columns=headers)
df1.head()

,Year,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1980,Kareem Abdul-Jabbar*,C,32,LAL,82,,38.3,10.2,16.9,...,.765,2.3,8.5,10.8,4.5,1.0,3.4,3.6,2.6,24.8
1,1980,Tom Abernethy,PF,25,GSW,67,,18.2,2.3,4.7,...,.683,0.9,1.9,2.9,1.3,0.5,0.2,0.6,1.8,5.4
2,1980,Alvan Adams,C,25,PHO,75,,28.9,6.2,11.7,...,.797,2.1,6.0,8.1,4.3,1.4,0.7,2.9,3.2,14.9
3,1980,Tiny Archibald*,PG,31,BOS,80,80,35.8,4.8,9.9,...,.830,0.7,1.7,2.5,8.4,1.3,0.1,3.0,2.7,14.1
4,1980,Dennis Awtrey,C,31,CHI,26,,21.5,1.0,2.3,...,.640,1.1,3.3,4.4,1.5,0.5,0.6,1.0,2.5,3.3


In [14]:
df1 = df1.drop_duplicates(subset='Player', keep='first')
rookies = df1[df1['Year'] != '1980']
rookies.head()

,Year,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
362,1981,Darrell Allums,PF,22,DAL,22,,12.5,1.0,3.0,...,.591,0.9,2.1,3.0,1.1,0.2,0.4,1.0,2.3,2.7
369,1981,Mel Bennett,PF,26,UTA,28,,11.2,0.9,2.1,...,.654,1.2,2.1,3.3,0.5,0.1,0.4,1.1,2.0,3.8
374,1981,Norman Black,SG,23,DET,3,,9.3,1.0,3.3,...,.250,0.0,0.7,0.7,0.7,0.3,0.0,0.3,0.7,2.7
385,1981,Dave Britton,PG,22,WSB,2,,4.5,1.0,1.5,...,,0.0,1.0,1.0,1.5,0.5,0.0,1.0,1.0,2.0
386,1981,Michael Brooks,SF,22,SDC,82,,30.2,6.0,12.4,...,.706,2.6,2.8,5.4,2.5,1.2,0.4,2.0,2.9,14.7


In [15]:
rookies = rookies.set_index('Player')
rookies2 = rookies.drop(['Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P','3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB','DRB', 'TOV', 'PF'], axis=1)
rookies2.head()

,Year,Pos,Age,TRB,AST,STL,BLK,PTS
Player,,,,,,,,
Darrell Allums,1981,PF,22,3.0,1.1,0.2,0.4,2.7
Mel Bennett,1981,PF,26,3.3,0.5,0.1,0.4,3.8
Norman Black,1981,SG,23,0.7,0.7,0.3,0.0,2.7
Dave Britton,1981,PG,22,1.0,1.5,0.5,0.0,2.0
Michael Brooks,1981,SF,22,5.4,2.5,1.2,0.4,14.7


In [16]:
rookies2['Age'] = pd.to_numeric(rookies2['Age'], errors='coerce')
rookies2['Year'] = pd.to_numeric(rookies2['Year'], errors='coerce')

In [17]:
rookies2['NBATRB'] = rookies2['TRB']
rookies2['NBAAST'] = rookies2['AST']
rookies2['NBASTL'] = rookies2['STL']
rookies2['NBABLK'] = rookies2['BLK']
rookies2['NBAPTS'] = rookies2['PTS']
rookies2 = rookies2.drop(['TRB', 'AST', 'STL', 'BLK', 'PTS'], axis=1)
rookies2.head()

,Year,Pos,Age,NBATRB,NBAAST,NBASTL,NBABLK,NBAPTS
Player,,,,,,,,
Darrell Allums,1981,PF,22,3.0,1.1,0.2,0.4,2.7
Mel Bennett,1981,PF,26,3.3,0.5,0.1,0.4,3.8
Norman Black,1981,SG,23,0.7,0.7,0.3,0.0,2.7
Dave Britton,1981,PG,22,1.0,1.5,0.5,0.0,2.0
Michael Brooks,1981,SF,22,5.4,2.5,1.2,0.4,14.7


In [33]:
players = rookies2.index.values
print(len(players))

2773


In [34]:
maindata = []
for name in players:
    realname = name
    name = name.lower()
    name = name.replace(" ", "-")
    website_url = requests.get('https://www.sports-reference.com/cbb/players/'+ name +'-1.html')
    soup = BeautifulSoup(website_url.content,'lxml')
    soup.prettify()
    My_table = soup.find('table',{'id':'players_per_game'})
    if My_table is not None:
        maindata = maindata + [realname]
        tabledata = My_table.findAll('td')
        data = []
        for cell in tabledata:
            data.append(cell.get_text())
        maindata = maindata +data[-28:]

In [35]:
import numpy as np
maindata2 = np.array(maindata)
refined2 = np.reshape(maindata2, (-1, 29))

In [36]:
tableheader = My_table.findAll('th')
colheaders = []
count = 0
for item in tableheader:
    if (count < 29):
        colheaders.append(item.get_text())
        count=count+1
colheaders.remove('Season')
colheaders = ['Name'] + colheaders
print(colheaders)

['Name', 'School', 'Conf', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '2P', '2PA', '2P%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', '\xa0', 'SOS']


In [37]:
import pandas as pd
college = pd.DataFrame(refined2, columns=colheaders)
college.tail()

,Name,School,Conf,G,GS,MP,FG,FGA,FG%,2P,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,,SOS
0,Darrell Allums,,,,,,4.1,,6.10,UCLA,...,.688,3.3,0.5,0.3,0.3,1.1,1.5,3.8,,5.29
1,Mel Bennett,1.0,,2.4,,3.3,10.1,,0.95,Pitt,...,.682,10.2,1.0,,2.4,,3.3,10.1,,0.95
2,Norman Black,1.3,1.2,0.4,,2.6,15.5,,-2.66,St. Joseph's,...,.667,8.7,1.4,1.1,0.4,,3.1,16.6,,-1.10
3,Dave Britton,,,,,,12.0,,2.90,Texas A&M,...,.687,3.1,,,,,,11.1,,3.57
4,Michael Brooks,2.1,,,,,24.1,,-1.31,La Salle,...,.700,12.0,1.7,,,,1.2,23.1,,-1.28


In [50]:
college = college.drop(['\xa0'], axis=1)

In [59]:
col = college.columns
cols = []
for co in col:
    cols.append(co)
cols.remove('Name')
cols.remove('School')
cols.remove('Conf')
print(cols)

['G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '2P', '2PA', '2P%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'SOS']


In [60]:
for col in cols:
    college[col] = pd.to_numeric(college[col], errors='coerce')

In [67]:
college = college.dropna()
college = college.drop(['Conf'], axis=1)
college.head()

,Name,School,G,GS,MP,FG,FGA,FG%,2P,2PA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,SOS
573,Mike Higgins,Navy,60.0,2.0,9.0,1.1,2.3,0.464,0.9,1.7,...,0.6,0.9,1.5,0.2,0.2,0.2,0.7,1.0,3.2,-5.93
923,Ashraf Amaya,Southern Illinois,128.0,120.0,32.2,5.3,9.4,0.567,5.2,9.0,...,2.9,3.4,8.9,0.6,1.1,1.2,1.9,3.3,14.6,0.74
1050,James Cotton,Maryland-Eastern Shore,19.0,3.0,13.9,0.8,1.8,0.441,0.8,1.7,...,0.7,2.0,2.7,0.4,0.6,0.3,0.9,2.4,2.9,-7.38
1195,Eddie Robinson,Lamar,114.0,75.0,32.6,1.8,4.9,0.365,1.5,3.8,...,0.3,3.6,3.1,3.4,0.8,0.2,2.9,1.4,5.2,-7.20
1256,Gilbert Arenas,Arizona,70.0,33.0,30.5,5.6,12.1,0.466,4.1,7.9,...,0.8,2.8,3.8,2.2,1.9,0.2,2.7,2.4,15.8,11.13


In [69]:
college.to_csv('CollegeStatLog2.csv')

In [72]:
college.reset_index(inplace=True)

In [ ]:
# maybe add year later
#Also maybe add position

In [78]:
final = college.merge(rookies2, left_on='Name', right_on='Player')
final = final.drop(['index'], axis=1)
final = final.set_index('Name')
final.head()

,School,G,GS,MP,FG,FGA,FG%,2P,2PA,2P%,...,BLK,TOV,PF,PTS,SOS,NBATRB,NBAAST,NBASTL,NBABLK,NBAPTS
Name,,,,,,,,,,,,,,,,,,,,,
Mike Higgins,Navy,60.0,2.0,9.0,1.1,2.3,0.464,0.9,1.7,0.510,...,0.2,0.7,1.0,3.2,-5.93,0.4,0.3,0.2,0.2,1.3
Ashraf Amaya,Southern Illinois,128.0,120.0,32.2,5.3,9.4,0.567,5.2,9.0,0.581,...,1.2,1.9,3.3,14.6,0.74,5.6,0.6,0.4,0.2,6.3
James Cotton,Maryland-Eastern Shore,19.0,3.0,13.9,0.8,1.8,0.441,0.8,1.7,0.469,...,0.3,0.9,2.4,2.9,-7.38,0.7,0.0,0.1,0.1,2.7
Eddie Robinson,Lamar,114.0,75.0,32.6,1.8,4.9,0.365,1.5,3.8,0.403,...,0.2,2.9,1.4,5.2,-7.20,2.7,0.5,0.7,0.4,7.0
Gilbert Arenas,Arizona,70.0,33.0,30.5,5.6,12.1,0.466,4.1,7.9,0.523,...,0.2,2.7,2.4,15.8,11.13,2.8,3.7,1.5,0.2,10.9


In [80]:
cols = rookies2.columns
for col in cols:
    final[col] = pd.to_numeric(final[col], errors='coerce')
final.head()

,School,G,GS,MP,FG,FGA,FG%,2P,2PA,2P%,...,BLK,TOV,PF,PTS,SOS,NBATRB,NBAAST,NBASTL,NBABLK,NBAPTS
Name,,,,,,,,,,,,,,,,,,,,,
Mike Higgins,Navy,60.0,2.0,9.0,1.1,2.3,0.464,0.9,1.7,0.510,...,0.2,0.7,1.0,3.2,-5.93,0.4,0.3,0.2,0.2,1.3
Ashraf Amaya,Southern Illinois,128.0,120.0,32.2,5.3,9.4,0.567,5.2,9.0,0.581,...,1.2,1.9,3.3,14.6,0.74,5.6,0.6,0.4,0.2,6.3
James Cotton,Maryland-Eastern Shore,19.0,3.0,13.9,0.8,1.8,0.441,0.8,1.7,0.469,...,0.3,0.9,2.4,2.9,-7.38,0.7,0.0,0.1,0.1,2.7
Eddie Robinson,Lamar,114.0,75.0,32.6,1.8,4.9,0.365,1.5,3.8,0.403,...,0.2,2.9,1.4,5.2,-7.20,2.7,0.5,0.7,0.4,7.0
Gilbert Arenas,Arizona,70.0,33.0,30.5,5.6,12.1,0.466,4.1,7.9,0.523,...,0.2,2.7,2.4,15.8,11.13,2.8,3.7,1.5,0.2,10.9


In [82]:
final = final.dropna()
final.to_csv('CollegeRookieStatLog.csv')

In [6]:
df1.head()

,Year,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1980,Kareem Abdul-Jabbar*,C,32,LAL,82,,38.3,10.2,16.9,...,.765,2.3,8.5,10.8,4.5,1.0,3.4,3.6,2.6,24.8
1,1980,Tom Abernethy,PF,25,GSW,67,,18.2,2.3,4.7,...,.683,0.9,1.9,2.9,1.3,0.5,0.2,0.6,1.8,5.4
2,1980,Alvan Adams,C,25,PHO,75,,28.9,6.2,11.7,...,.797,2.1,6.0,8.1,4.3,1.4,0.7,2.9,3.2,14.9
3,1980,Tiny Archibald*,PG,31,BOS,80,80,35.8,4.8,9.9,...,.830,0.7,1.7,2.5,8.4,1.3,0.1,3.0,2.7,14.1
4,1980,Dennis Awtrey,C,31,CHI,26,,21.5,1.0,2.3,...,.640,1.1,3.3,4.4,1.5,0.5,0.6,1.0,2.5,3.3


In [7]:
df2 = pd.read_csv('CollegeRookieStatLog.csv')
df2.head()

,Name,School,G,GS,MP,FG,FGA,FG%,2P,2PA,...,BLK,TOV,PF,PTS,SOS,NBATRB,NBAAST,NBASTL,NBABLK,NBAPTS
0,Mike Higgins,Navy,60.0,2.0,9.0,1.1,2.3,0.464,0.9,1.7,...,0.2,0.7,1.0,3.2,-5.93,0.4,0.3,0.2,0.2,1.3
1,Ashraf Amaya,Southern Illinois,128.0,120.0,32.2,5.3,9.4,0.567,5.2,9.0,...,1.2,1.9,3.3,14.6,0.74,5.6,0.6,0.4,0.2,6.3
2,James Cotton,Maryland-Eastern Shore,19.0,3.0,13.9,0.8,1.8,0.441,0.8,1.7,...,0.3,0.9,2.4,2.9,-7.38,0.7,0.0,0.1,0.1,2.7
3,Eddie Robinson,Lamar,114.0,75.0,32.6,1.8,4.9,0.365,1.5,3.8,...,0.2,2.9,1.4,5.2,-7.20,2.7,0.5,0.7,0.4,7.0
4,Gilbert Arenas,Arizona,70.0,33.0,30.5,5.6,12.1,0.466,4.1,7.9,...,0.2,2.7,2.4,15.8,11.13,2.8,3.7,1.5,0.2,10.9


In [9]:
df2.columns

Index(['Name', 'School', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '2P', '2PA',
       '2P%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'SOS', 'NBATRB', 'NBAAST',
       'NBASTL', 'NBABLK', 'NBAPTS'],
      dtype='object')

In [18]:
df2 = df2.drop(['NBATRB', 'NBAAST', 'NBASTL', 'NBABLK', 'NBAPTS'], axis=1)
df2.head()

,Name,School,G,GS,MP,FG,FGA,FG%,2P,2PA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,SOS
0,Mike Higgins,Navy,60.0,2.0,9.0,1.1,2.3,0.464,0.9,1.7,...,0.6,0.9,1.5,0.2,0.2,0.2,0.7,1.0,3.2,-5.93
1,Ashraf Amaya,Southern Illinois,128.0,120.0,32.2,5.3,9.4,0.567,5.2,9.0,...,2.9,3.4,8.9,0.6,1.1,1.2,1.9,3.3,14.6,0.74
2,James Cotton,Maryland-Eastern Shore,19.0,3.0,13.9,0.8,1.8,0.441,0.8,1.7,...,0.7,2.0,2.7,0.4,0.6,0.3,0.9,2.4,2.9,-7.38
3,Eddie Robinson,Lamar,114.0,75.0,32.6,1.8,4.9,0.365,1.5,3.8,...,0.3,3.6,3.1,3.4,0.8,0.2,2.9,1.4,5.2,-7.20
4,Gilbert Arenas,Arizona,70.0,33.0,30.5,5.6,12.1,0.466,4.1,7.9,...,0.8,2.8,3.8,2.2,1.9,0.2,2.7,2.4,15.8,11.13


In [20]:
final = df2.merge(rookies2, left_on='Name', right_on='Player')
#final = final.drop(['index'], axis=1)
final = final.set_index('Name')
final.head()

,School,G,GS,MP,FG,FGA,FG%,2P,2PA,2P%,...,PTS,SOS,Year,Pos,Age,NBATRB,NBAAST,NBASTL,NBABLK,NBAPTS
Name,,,,,,,,,,,,,,,,,,,,,
Mike Higgins,Navy,60.0,2.0,9.0,1.1,2.3,0.464,0.9,1.7,0.510,...,3.2,-5.93,1990,SF,22,0.4,0.3,0.2,0.2,1.3
Ashraf Amaya,Southern Illinois,128.0,120.0,32.2,5.3,9.4,0.567,5.2,9.0,0.581,...,14.6,0.74,1996,PF,24,5.6,0.6,0.4,0.2,6.3
James Cotton,Maryland-Eastern Shore,19.0,3.0,13.9,0.8,1.8,0.441,0.8,1.7,0.469,...,2.9,-7.38,1998,SG,22,0.7,0.0,0.1,0.1,2.7
Eddie Robinson,Lamar,114.0,75.0,32.6,1.8,4.9,0.365,1.5,3.8,0.403,...,5.2,-7.20,2000,SF,23,2.7,0.5,0.7,0.4,7.0
Gilbert Arenas,Arizona,70.0,33.0,30.5,5.6,12.1,0.466,4.1,7.9,0.523,...,15.8,11.13,2002,PG,20,2.8,3.7,1.5,0.2,10.9


In [21]:
final.to_csv('CollegeRookieStatLog2.csv')

In [118]:
#maybe combine with dummified college name too?